# 13주 과제
블록체인에서 주문을 하고, 그 내역을 로컬 파일에 작성해 보자.

블록체인으로 주문을 전송하면, 이벤트가 발생하고 주문내역을 로컬파일에 쓰게 된다.

* EventTest.sol에서는 이벤트가 발생하면서 그 로그를 파일에 작성한다 (OrderEvent.txt)

* OrderEvent.sol을 수정하여:

	* order(상품항목, 주문 개수, 배송지) 함수 작성

	* getBalance() 함수 작성. 해당 프로그램 생성한 Owner만 읽을 수 있게 modfier isOnwer 작성.



geth 또는 ganache 어느 하나를 선택하여 프로그램을 작성하세요.

아래 로그가 파일에 쓰여지도록 하고, 노트북에서 그 파일을 출력하세요 (!type src/OrderEvent.txt)

전송자주소, 1111, 3, 20 2-gil Hongji-dong Jongro-gu Seoul

전송자주소, 1111, 5, 20 2-gil Hongji-dong Jongro-gu Seoul

전송자주소, 1111, 20, 20 2-gil Hongji-dong Jongro-gu Seoul

# 1단계 : 컨트랙 개발

In [1]:
%%writefile src/BlockOrder.sol

pragma solidity ^0.6.0;

contract OrderEvent {
    address owner;
    event OrderLog(address indexed, uint, uint, string); 
    
    constructor() public{
        owner = msg.sender;
    }
    
    modifier isOwner() { 
        if (msg.sender != owner) {
            revert(); // exception이 발생해서 중단하는 것
        }
        _; 
    }

    function order(uint _itemId, uint amount, string memory _from) public payable { 
        emit OrderLog(msg.sender,_itemId, amount, _from);
    }
    
    function getBalance() view public isOwner returns(uint) { // 오직 Owner만 볼 수 있는 잔고
        return address(this).balance;
    }
}


Overwriting src/BlockOrder.sol


# 2단계 : 컴파일

In [2]:
!solc --abi --bin --gas src/BlockOrder.sol


======= src/BlockOrder.sol:OrderEvent =======
Gas estimation:
construction:
   21043 + 129400 = 150443
external:
   getBalance():	1063
   order(uint256,uint256,string):	infinite
Binary:
608060405234801561001057600080fd5b50336000806101000a81548173ffffffffffffffffffffffffffffffffffffffff021916908373ffffffffffffffffffffffffffffffffffffffff160217905550610287806100606000396000f3fe6080604052600436106100295760003560e01c806312065fe01461002e578063b0bb49eb14610059575b600080fd5b34801561003a57600080fd5b50610043610128565b6040518082815260200191505060405180910390f35b6101266004803603606081101561006f57600080fd5b810190808035906020019092919080359060200190929190803590602001906401000000008111156100a057600080fd5b8201836020820111156100b257600080fd5b803590602001918460018302840111640100000000831117156100d457600080fd5b91908080601f016020809104026020016040519081016040528093929190818152602001838380828437600081840152601f19601f820116905080830192505050505050509192919290505050610189565b005b60008060009054906101000a900

# 3단계 : 컨트랙 배포

In [3]:
%%writefile src/BlockOrderDeploy.js

var Web3=require('web3');
var web3 = new Web3(new Web3.providers.HttpProvider("http://127.0.0.1:8345"));
var _abiArray =[{"inputs":[],"stateMutability":"nonpayable","type":"constructor"},{"anonymous":false,"inputs":[{"indexed":true,"internalType":"address","name":"","type":"address"},{"indexed":false,"internalType":"uint256","name":"","type":"uint256"},{"indexed":false,"internalType":"uint256","name":"","type":"uint256"},{"indexed":false,"internalType":"string","name":"","type":"string"}],"name":"OrderLog","type":"event"},{"inputs":[],"name":"getBalance","outputs":[{"internalType":"uint256","name":"","type":"uint256"}],"stateMutability":"view","type":"function"},{"inputs":[{"internalType":"uint256","name":"_itemId","type":"uint256"},{"internalType":"uint256","name":"amount","type":"uint256"},{"internalType":"string","name":"_from","type":"string"}],"name":"order","outputs":[],"stateMutability":"payable","type":"function"}];
//var _abiArray = JSON.parse(_abiStr);
var _bin="0x" + "608060405234801561001057600080fd5b50336000806101000a81548173ffffffffffffffffffffffffffffffffffffffff021916908373ffffffffffffffffffffffffffffffffffffffff160217905550610287806100606000396000f3fe6080604052600436106100295760003560e01c806312065fe01461002e578063b0bb49eb14610059575b600080fd5b34801561003a57600080fd5b50610043610128565b6040518082815260200191505060405180910390f35b6101266004803603606081101561006f57600080fd5b810190808035906020019092919080359060200190929190803590602001906401000000008111156100a057600080fd5b8201836020820111156100b257600080fd5b803590602001918460018302840111640100000000831117156100d457600080fd5b91908080601f016020809104026020016040519081016040528093929190818152602001838380828437600081840152601f19601f820116905080830192505050505050509192919290505050610189565b005b60008060009054906101000a900473ffffffffffffffffffffffffffffffffffffffff1673ffffffffffffffffffffffffffffffffffffffff163373ffffffffffffffffffffffffffffffffffffffff161461018357600080fd5b47905090565b3373ffffffffffffffffffffffffffffffffffffffff167f40c379409af56ac92abea08a48623434cb7ac020221d5e4840d056e19d6ec2958484846040518084815260200183815260200180602001828103825283818151815260200191508051906020019080838360005b838110156102105780820151818401526020810190506101f5565b50505050905090810190601f16801561023d5780820380516001836020036101000a031916815260200191505b5094505050505060405180910390a250505056fea264697066735822122042b8eb617c5ae5ea67f3bb2fc4bc6b51baabb268ee3f5092fb15f94e2c8af88f64736f6c63430006030033";

async function deploy() { 
    const accounts = await web3.eth.getAccounts();
    console.log("Deploying the contract from " + accounts[0]);
    var deployed = await new web3.eth.Contract(_abiArray)
        .deploy({data: _bin})
        .send({from: accounts[0], gas: 300000}, function(err, transactionHash) {
                if(!err) console.log("hash: " + transactionHash); 
        })
        //.then(function(newContractInstance) {
        //    console.log(newContractInstance.options.address)
        //});
    console.log("---> The contract deployed to: " + deployed.options.address)
}

deploy()

Overwriting src/BlockOrderDeploy.js


In [4]:
!node src/BlockOrderDeploy.js

Deploying the contract from 0xCeC08a4840d7F599FDECDE5E553e95aeFc4251Bd
hash: 0xd24c993ef80a6ea8e45caf6bd84661101f1fd5e51b40190e03861ff5f7e4cc98
---> The contract deployed to: 0xB159a9389763299B0920BF44e315E36e3f695Cf1


# 4단계 : 사용

In [5]:
%%writefile src/BlockOrderUse.js

var fs=require('fs');
var Web3=require('web3');
var web3;
if (typeof web3 !== 'undefined') {
    web3 = new Web3(web3.currentProvider);
} else {
    var web3 = new Web3(new Web3.providers.WebsocketProvider("http://127.0.0.1:8345"));
}


var _abiArray=[{"inputs":[],"stateMutability":"nonpayable","type":"constructor"},{"anonymous":false,"inputs":[{"indexed":true,"internalType":"address","name":"","type":"address"},{"indexed":false,"internalType":"uint256","name":"","type":"uint256"},{"indexed":false,"internalType":"uint256","name":"","type":"uint256"},{"indexed":false,"internalType":"string","name":"","type":"string"}],"name":"OrderLog","type":"event"},{"inputs":[],"name":"getBalance","outputs":[{"internalType":"uint256","name":"","type":"uint256"}],"stateMutability":"view","type":"function"},{"inputs":[{"internalType":"uint256","name":"_itemId","type":"uint256"},{"internalType":"uint256","name":"amount","type":"uint256"},{"internalType":"string","name":"_from","type":"string"}],"name":"order","outputs":[],"stateMutability":"payable","type":"function"}];
async function doIt() {
    var _order = new web3.eth.Contract(_abiArray, '0xB159a9389763299B0920BF44e315E36e3f695Cf1');
    const accounts = await web3.eth.getAccounts();
    console.log("Account: " + accounts[0]);
    var event = _order.events.OrderLog({//7
            //filter: {_from: accounts[0], _value: 20}, //주소를 넣음 6
            fromBlock: 'latest', toBlock: 'pending'
        }, function (error, result) { //7,5
        if (!error) { 
            log = JSON.stringify(result.returnValues);
            console.log("Event fired: " + log);
            {//3
                fs.appendFile("src/OrderEvent.txt", log, "utf-8", function(e){//2
                if(!e){//1
                    console.log(">> Writing to file");
                }//1
            }//2
        )}//3
        }//4
    });//5
    
    var value;
    const balanceBefore = await web3.eth.getBalance(accounts[0]);
    console.log("Balance before: " + balanceBefore);
    my = await _order.methods.order(1111, 3,"20 2-gil Hongji-dong Jongro-gu Seoul") //여기서 불러주는 중, 리믹스에서 쓴 것처럼 쓴다
        .send({from: accounts[0], gas: 100000})
    console.log("---> MyFunction called " + JSON.stringify(my.events.OrderLog.returnValues));
    
    my = await _order.methods.order(1111, 5,"20 2-gil Hongji-dong Jongro-gu Seoul") //여기서 불러주는 중, 리믹스에서 쓴 것처럼 쓴다
        .send({from: accounts[0], gas: 100000})
    console.log("---> MyFunction called " + JSON.stringify(my.events.OrderLog.returnValues));
    
    my = await _order.methods.order(1111, 20,"20 2-gil Hongji-dong Jongro-gu Seoul") //여기서 불러주는 중, 리믹스에서 쓴 것처럼 쓴다
        .send({from: accounts[0], gas: 100000})
    console.log("---> MyFunction called " + JSON.stringify(my.events.OrderLog.returnValues));
    
    const balanceAfter = await web3.eth.getBalance(accounts[0]);
    console.log("Balance after: " + balanceAfter);
    console.log("Balance diff: " + (balanceBefore - balanceAfter));
    process.exit(1);   
    }

doIt()

Overwriting src/BlockOrderUse.js


In [6]:
!node src/BlockOrderUse.js

Account: 0xCeC08a4840d7F599FDECDE5E553e95aeFc4251Bd
Balance before: 99995703220000000000
Event fired: {"0":"0xCeC08a4840d7F599FDECDE5E553e95aeFc4251Bd","1":"1111","2":"3","3":"20 2-gil Hongji-dong Jongro-gu Seoul"}
>> Writing to file
---> MyFunction called {"0":"0xCeC08a4840d7F599FDECDE5E553e95aeFc4251Bd","1":"1111","2":"3","3":"20 2-gil Hongji-dong Jongro-gu Seoul"}
Event fired: {"0":"0xCeC08a4840d7F599FDECDE5E553e95aeFc4251Bd","1":"1111","2":"5","3":"20 2-gil Hongji-dong Jongro-gu Seoul"}
>> Writing to file
---> MyFunction called {"0":"0xCeC08a4840d7F599FDECDE5E553e95aeFc4251Bd","1":"1111","2":"5","3":"20 2-gil Hongji-dong Jongro-gu Seoul"}
Event fired: {"0":"0xCeC08a4840d7F599FDECDE5E553e95aeFc4251Bd","1":"1111","2":"20","3":"20 2-gil Hongji-dong Jongro-gu Seoul"}
>> Writing to file
---> MyFunction called {"0":"0xCeC08a4840d7F599FDECDE5E553e95aeFc4251Bd","1":"1111","2":"20","3":"20 2-gil Hongji-dong Jongro-gu Seoul"}
Balance after: 99994136140000000000
Balance diff: 1567079999995904

In [7]:
!type C:\Users\15Z970-G.AA5BK\Code\jsr\src\OrderEvent.txt

{"0":"0xCeC08a4840d7F599FDECDE5E553e95aeFc4251Bd","1":"1111","2":"3","3":"20 2-gil Hongji-dong Jongro-gu Seoul"}{"0":"0xCeC08a4840d7F599FDECDE5E553e95aeFc4251Bd","1":"1111","2":"5","3":"20 2-gil Hongji-dong Jongro-gu Seoul"}{"0":"0xCeC08a4840d7F599FDECDE5E553e95aeFc4251Bd","1":"1111","2":"20","3":"20 2-gil Hongji-dong Jongro-gu Seoul"}
